In [14]:
import cartpole as cart

In [15]:
cart.simulate(10, True)

episode 0 reached 500 step.


FileNotFoundError: [Errno 2] No such file or directory: './data/ep0_state_test.npy'

In [16]:
cart.test()


AttributeError: module 'cartpole' has no attribute 'test'